# <center>The Path Planning Algorithm (PPA)</center>
    Luis Robaina : luis.f.robaina@nasa.gov
    Ames Reseach Center Intern, Fall 2020.
    Mentored by: Gilbert Wu, Tech Lead Modeling and Simulation.
                 gilbert.wu@nasa.gov
    
    Last Update: Nov 8, 2020.
    
**Note: This Notebook is for documentation purposes, to train/test the PPA please use the source code and refer to  README for instructions.**

### Introduction
As we find more potential applications to Unmanned Aircraft Systems (UAS), the ability to safely operate them has become an important challenge. The demand for UAS has steadily increased during the last 10 years. Our capacity to safely integrate these unmanned aircraft with manned aircraft in the airspace requires the ability to equip them with both collision avoidance (CA) and detect and avoid (DAA) systems capabilities. DAA systems use real-time telemetry to detect other aircraft in their vicinity and provide the UAS operator with the necessary guidance to be aware of collision risk and to collaborate with air traffic control to address the risk. It is also necessary to identify potential conflict-free trajectories so that the UAS can recover its flight path. 
Significant efforts are currently in progress to find algorithms that efficiently replicate the functionality of DAA systems. Previous efforts to construct a robust DAA system have been based on geometric formulations as well as Deep Learning approaches. The approach taken in this research project is known as Reinforcement Learning (RL) — A form of Machine Learning (ML). This approach aims to construct a proof-of-concept RL model able to recommend robust maneuver strategies for a UAS in conflict with another aircraft under simplified conditions. Compared to other formulations of this problem, this approach does not require labeled training data — commonly used in supervised ML — or complex geometric algorithms to achieve its objectives. This approach lets the UAS interact with a simulated environment to generate its training data: Rewarding sequence of maneuvers that create successful trajectories and penalizing those maneuvers that generate some form of conflict. Running numerous simulations over a set of different encounter geometries with this RL process, the UAS can find a trajectory without conflicts to the destination. 


## <center>Sample Conflict Resolution Strategy</center>

![Sample Result](Images/2.jpg "PPA Encounter Solution")

## <center>PPA Source Code Structure</center>

![Project Modules Graph](Images/PPAGraph.png)

### Abbreviations and Terms Used
- **PPA** : Path Planning Algorithm
- **Ownship** : The aircraft that this algorithm controls, aka "The agent"
- **Intruder** : The aircraft that the ownship will avoid conflict with. This algorithm does not control the intruder
- **LODWC** : Lost of Distance "Well Clear"
- **CPA** : Closest Point of Approach
- **MCTS** : Monte Carlo Tree Search
- **The Model** : Refers to the file that will store the results of our training and will be used to test. The model answers the following question: I am at this discrete state, what is the best action to take? The model is created during training and loaded during testing.
- **Q Value** : "The expected reward". In the context of MCTS this is the expected discounted sum of rewards that one can expect starting from a given node on the tree down its sub-tree. In the context of the model this is the expected reward for taking an action from a given discrete state. 

In [10]:
# Modules: Remember to run pip install before you run the source code :)
import math
from random import random
import numpy as np
from numpy import linalg as LA
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd
import pickle as p
import os
import shutil

## <center>PPA Algorithm Constants</center>

<center>Source Code Module: <i>global_constants.py<i></center>
    
**<u>Notes About Global Constants:</u>**
    
- **TIME_INCREMENT**: Every action is assumed to take <i>TIME_INCREMENT</i> senconds once it is selected by the agent (During training), Directly affects the performance of the algorithm in terms of training time and coverage of the state. Small time increments during training requires more training time to cover the state space.
- **TEST_TIME_INCREMENT**: Similar to TIME_INCREMENT but used only when testing the PPA (Not during training).
- **GAMMA**: A discount factor for future actions: Actions that lead to a reward after many steps are valued less than the same rewards that come by fewer steps. [Article about Gamma](https://towardsdatascience.com/penalizing-the-discount-factor-in-reinforcement-learning-d672e3a38ffe)
- **DESTINATION_STATE**: The coordinates of the destination state for the problem definition, for all encounters we make the destination be the origin of the 2D coordinate system and place the ownship on the negative y axis at some distance.
- **DESTINATION_STATE_REWARD**: This is the only reward our agent gets, if a sequence of actions leads to the agent arriving to the destination then those actions are rewarded by +1.
- **ABANDON_STATE_REWARD**: Penalty if a sequence of actions lead to the agent being too far from the destination.
- **ABANDON_STATE_ERROR**: If the agent is at a distance greated than or equal to the value of this constant we consider it an Abandon State and penalize it with ABANDON_STATE_REWARD.
- **LODWC_REWARD**: Penalty if a sequence of actions lead to the agent being too close to the intruder (LODWC).
- **DWC_DIST**: If this minimum distance between ownship and intruder is violated we consider it a LODWC.  
- **TURN_ACTION_REWARD**: Penalty for every turn action the agent selects: Turning is resource inneficient, if a conflict can be avoided simply by going straight then avoid turnig or will penalize the agent.

In [11]:
import os

# Algorithm's Constants:
# Refer to README for more details about rewards/penalties.
TIME_INCREMENT = 10.0           # Seconds that each action will runs for (During training)
TEST_TIME_INCREMENT = 10.0      # Seconds that each action will run for (During testing)
DESTINATION_STATE = [0, 0]      # Coordinates of the destination.

DESTINATION_STATE_REWARD = 1.0  # Reward for reaching the destination.
ABANDON_STATE_REWARD = -0.5     # Negative reward (a penalty) if ownship reaches state too far from destination.
# We define Lost of Well Clear if the distance between aircraft is less than 2200 ft.
LODWC_REWARD = -0.3             # Negative reward (penalty) for Lost of Well Clear.
TURN_ACTION_REWARD = -0.00001   # Negative reward (penalty) for every turn action.

# Final State Constants:
DWC_DIST = 2200                 # (ft) Well Clear distance.

DESTINATION_DIST_ERROR = 500    # (ft) Max distance from the destination which is considered Destination Reached.

ABANDON_STATE_ERROR = 30000     # (ft) Distance which if exceed results on an abandon state.

# Set of actions the ownship can take.
ACTIONS = {                     # Actions are in degrees per second.
    'LEFT': -5,
    'NO_TURN': 0,
    'RIGHT': 5
}

"""
Maximum number of MCTS iterations that can run for a given encounter. 
Each iteration of MCTS includes: selection, expansion, simulation.
Refer to MCTS.py for more details.
"""
MCTS_ITERATIONS = 10000
# Every MCTS_CUT iterations try to construct a trajectory. If it is successful then move to the next training encounter.
# If a cut is not desired set MCTS_CUT = 1. Then every MCTS will go for MCTS_ITERATIONS.
MCTS_CUT = 1000

UCB1_C = 2                      # UCB1 Exploration term.
GAMMA = 0.9                     # Discount Factor.

# Max number of actions that can be taken when simulating for Performance.
EPISODE_LENGTH = None

# Directory Paths:
# Set of Training Encounters.
TRAINING_SET = 'Test Files/Test_Encounter_Geometries.csv'
# Directory where each encounter description/trajectory will be placed.
TEST_RESULTS_PATH = os.getcwd() + '/Test Results'

# Conversion Factors
NMI_TO_FT = 6076.12
HR_TO_SEC = 3600

# For state discretization purposes:
MIN_DISTANCE = 0              # (ft).
MAX_DISTANCE = 60761          # (ft) equivalent to 10 Nautical Miles.

MIN_SPEED = 0                 # (ft/sec).
MAX_SPEED = 287               # About 170 knot (ft/sec).

MIN_ANGLE = -180              # (deg).
MAX_ANGLE = 180               # (deg).

# <center>Load a Set of Test Encounter Geometries</center>
<a id='encounter_properties'></a>

**Take a look at the way the encounter geometries are described with this sample training set:**

In [12]:
ENCOUNTERS_GEOMETRIES = pd.read_csv(TRAINING_SET, header = 0)
ENCOUNTERS_GEOMETRIES


,Run,time_to_CPA_sec,destination_time_after_CPA_sec,OIF_CPA,CPA_distance_ft,v_o_kts,v_i_kts,int_rel_heading_deg
0,0,90,30,False,2000,100,0,0
1,1,90,30,False,1000,100,0,0
2,2,90,30,True,0,100,0,0
3,3,90,30,True,1000,100,0,0
4,4,90,30,True,2000,100,0,0
...,...,...,...,...,...,...,...,...
275,275,90,30,False,2000,100,170,-180
276,276,90,30,False,1000,100,170,-180
277,277,90,30,True,0,100,170,-180
278,278,90,30,True,1000,100,170,-180


## Training Encounter Geometry Features
- **Run**: Index of a given training encounter (Must start at 0).
- **Time to CPA (sec)** : The number of seconds that the ownship takes to reach the closest point of approximation (CPA) moving at a constant velocity without any manouvers.
(The "Closest Point of Approach" refers to the positions at which two dynamically moving objects reach their closest possible distance)
- **Destination time after CPA**: The number of seconds the ownship takes to reach the destination point after the CPA.

**(The sum of Time to CPA and Destination time after CPA allows us to place the ownship on the exact coordinates such that it takes the sum of these times to reach destination point at the given speed without any manouvers)**
- **OIF_CPA**: True if the ownship passes in front of  the intruder at CPA, false otherwise.
- **CPA_distance_ft**: Distance between the ownship and intruder at CPA in feet.
- **v_o_kts and v_i_kts**: Speed of the ownship and intruder respectively in Knot: The knot is a unit of speed equal to one nautical mile per hour.
- **int_rel_heading_deg**: Intruder's heading relative to the ownship's initial heading.

### Absolute State Definition
Defines the state of the ownship-intruder system in continuous 2D coordinates.
<a id='absolute_state'></a>

In [13]:
class State:
    # Define a state object type.
    def __init__(self, ownship_pos, intruder_pos, ownship_vel, intruder_vel):

        """
        Note:
            All the features of a State object are numpy arrays.
        """
        self.ownship_pos = ownship_pos      # [x,y] (ft)
        self.intruder_pos = intruder_pos    # [x,y] (ft)
        self.ownship_vel = ownship_vel      # [v_x,v_y] (ft/sec)
        self.intruder_vel = intruder_vel    # [v_x,v_y] (ft/sec)

    def get_distance(self):                 # Distance between the aircraft in ft.
        return LA.norm(self.ownship_pos - self.intruder_pos)
    
    def __str__(self):                      # String representation of a State object.
        return f"""
            own_pos (ft) = [{self.ownship_pos}]
            own_vel (ft/s) = [{self.ownship_vel}]
            int_pos (ft) = [{self.intruder_pos}]
            int_vel (ft/s) = [{self.intruder_vel}]        
        """

## Compute Initial State.
**Creates an [ABSOLUTE STATE](#absolute_state) object from a given record of the [ENCOUNTER GEOMETRIES TABLE](#encounter_properties).**

Assumptions:
1. Ownship always heads north, $ v_{x_{0}} = 0$. 

2. Origin is always at the destination point $[0,0]$

In [14]:
def computeInitialState(encounter_properties: dict) -> State:
    """
        Compute the ownship and intruder's initial states based on the encounter
        design parameters:
        encounter_properties is a dictionary as follows:
            encounter_properties = {
                0: (time_to_CPA),
                1: (destination_time_after_CPA)
                2: (OIF_CPA),
                3: (CPA_distance_ft),
                4: (v_o_kts)
                5: (v_i_kts),
                6: (int_rel_heading_deg),
                7: (total_runs),
                8: (depth),
                9: (skip)
            }
    """
    
    """
        For the ownship:
    """
    # Velocity of the ownship.
    # Note: Ownship flights north so the x component of ownship_vel is 0.
    ownship_vel_x = 0   # (ft/s).
    speed_ownship = float(encounter_properties[4])
    ownship_vel_y = speed_ownship * NMI_TO_FT / HR_TO_SEC   # (ft/s).
    
    # Velocity vector for the ownship. 
    ownship_vel = np.array([ownship_vel_x, ownship_vel_y])
    
    # Position of ownship.
    # Ownship is placed south of the destination point [0,0].
    ownship_x = 0
    
    # Using time to CPA and time to destination after CPA we can compute the initial y coordinates for the ownship.
    time_to_CPA = float(encounter_properties[0])
    destination_time_after_CPA = float(encounter_properties[1])
    # Place the ownship at the correct y coordinate such that at the given velocity it will take 
    # (time_to_CPA + destination_time_after_CPA) seconds to reach the destination [0,0].
    ownship_y = -(time_to_CPA + destination_time_after_CPA) * ownship_vel_y # (ft).
    
    # position vector (ownship).
    ownship_pos = np.array([ownship_x, ownship_y])
    
    """
        For the intruder:
    """
    # Velocity of the intruder:
    
    intruder_velocity_magnitud = float(encounter_properties[5]) * NMI_TO_FT / HR_TO_SEC     # (ft/s).
    intruder_heading_angle = float(encounter_properties[6])     # degrees.
    
    intruder_vel_x = intruder_velocity_magnitud * math.sin(math.radians(intruder_heading_angle))
    intruder_vel_y = intruder_velocity_magnitud * math.cos(math.radians(intruder_heading_angle))
    
    # velocity vector (intruder). 
    intruder_vel = np.array([intruder_vel_x, intruder_vel_y])
    
    # Position of the intruder:
    
    # Solve for initial position difference vector, delta_pos_t0:
    delta_vel_t0 = intruder_vel - ownship_vel
    delta_vel_magnitud = LA.norm(delta_vel_t0)
    
    # Get the horizontal distance at CPA.
    S = float(encounter_properties[3])
    # Initial distance btw aircraft:
    delta_pos_magnitud = math.sqrt((S**2) + (time_to_CPA**2 * delta_vel_magnitud**2))
    
    # Let the angle between delta_vel_t0 and delta_post_t0 be theta.
    cos_theta = -time_to_CPA * math.sqrt(delta_vel_magnitud**2) / delta_pos_magnitud
    sin_theta_2 = 1 - cos_theta**2
    
    if sin_theta_2 < 0 and sin_theta_2 > -1e-15:
        sin_theta_2 = 0
    
    sin_theta = math.sqrt(sin_theta_2)

    # Given the angle we can rotate delta_vel_t0 to get delta_post_t0:
    # Clock-wise rotation.
    rotation_matrix = np.array([
        [cos_theta, -sin_theta],
        [sin_theta, cos_theta]
    ])
    delta_post_t0 = (rotation_matrix@delta_vel_t0)*delta_pos_magnitud/delta_vel_magnitud
    
    # Assume position of ownship is [0,0] then position of intruder is delta_post_t0.
    # if x position of intruder at CPA is < 0 then ownship passes in front.
    
    # position vector (intruder).
    intruder_pos = ownship_pos + delta_post_t0
    intruder_pos_at_CPA = intruder_pos + (time_to_CPA*intruder_vel)
    
    OIF_CPA = bool(encounter_properties[2])
    
    if intruder_pos_at_CPA[0] <= 0 and OIF_CPA is True or intruder_pos_at_CPA[1] >= 0 and OIF_CPA is False:
        # Rotation performed was the correct rotation.
        pass

    else:
        # Perform Counter-clock wise rotation.
        rotation_matrix = np.array([ 
            [cos_theta, sin_theta],
            [-sin_theta,  cos_theta]
        ])
        delta_post_t0 = (rotation_matrix@delta_vel_t0)*delta_pos_magnitud/delta_vel_magnitud
        intruder_pos = ownship_pos + delta_post_t0
    
    # Create State object
    encounter_state = State(ownship_pos, intruder_pos, ownship_vel, intruder_vel)
    return encounter_state


**Compute an initial state object given a directory for the training encounter csv file and an index for an encounter tuple in the table.**

In [15]:
def getInitStateFromEncounter(encounter_directory, encounter_index):
    """
    Load the desc.csv file that contains the details about an encounter and get the initial state.

    :param encounter_directory:
    :param encounter_index:
    :return:
    """
    # Load an encounter description from the directory.
    ENCOUNTER_DESC = pd.read_csv(encounter_directory + '/desc.csv')

    # Convert the encounter to a dictionary.
    encounter_properties = ENCOUNTER_DESC.to_dict().get(str(encounter_index))

    """
    Note:
    encounter_properties is a dictionary with the following integer keys:
        0: (time_to_CPA_sec) 
        1: (destination_time_after_CPA_sec) 
        2: (OIF_CPA) 
        3: (CPA_distance_ft)
        4: (v_o_kts) 
        5: (v_i_kts) 
        6: (int_rel_heading_deg)
    """
    # Given the encounter properties, compute the initial state of the system of the two aircraft.
    encounter_state = computeInitialState(encounter_properties)
    return encounter_state

**Given a [ABSOLUTE STATE](#absolute_state) object and an action to take, compute the next state object after the action**<br>
**Given state $q$ and action $a$ take $(q,a) -> q'$. Where $q'$ is a new state in continious space.
Recall all rotation actions are $5$ deg/sec.**

In [16]:
def getNewState(state: State, action, TIME):
    """
        Returns an instance of State which
        represents a new state after taking an
        action: (q,a) -> q'
        TIME: How long should an action go for.

    """
    ownship_vel = np.array(state.ownship_vel)
    
    # Velocity:
    if action is 'NO_TURN':
        new_vel_own = ownship_vel   # [v_x,v_y] (ft/sec).

    else:
        theta = 5   # degrees.
        cos_theta = math.cos(math.radians(theta))
        sin_theta = math.sin(math.radians(theta))
        
        if action is 'LEFT':
            # Perform Counter-clock wise rotation.
            rotation_matrix = np.array([ 
                [cos_theta, sin_theta],
                [-sin_theta,  cos_theta]
            ])
            new_vel_own = rotation_matrix@ownship_vel

        elif action is 'RIGHT':
            # Perform clock-wise rotation.
            rotation_matrix = np.array([
                [cos_theta, -sin_theta],
                [sin_theta, cos_theta]
            ])
            new_vel_own = rotation_matrix@ownship_vel
    
    # Position:
    # For Ownship:
    avg_disp = 0.5 * (new_vel_own + ownship_vel) * TIME
    new_own_pos = state.ownship_pos + avg_disp  # [x_o,y_o] (ft).
    
    # For Intruder: Intruder flights at a constant velocity.
    intr_vel = np.array(state.intruder_vel)
    new_vel_intr = intr_vel
    intr_disp = 0.5 * (new_vel_intr + intr_vel) * TIME
    new_intr_pos = state.intruder_pos + intr_disp
    
    # New state after the action.
    new_state = State(new_own_pos, new_intr_pos, new_vel_own, new_vel_intr)
    return new_state

### Local State Definition
<a id='local_state'></a>
A Local State is a geometric representation of an Absolute State in continuous 2D coordinates. Converting an Absolute States to Local States allows us to take advantage of differnt forms of symmetry and simplify modeling.

In [17]:
class LocalState:
    """
    A Local State is a geometrical representation of a given state in continuous
    2D coordinates. Converting Absolute states to Local states allows us to take
    advantage of symmetry.
    """
    def __init__(self, r_do, theta_do, v_do, r_io, theta_io, psi_io_nr_io ,v_i ):
        # Distance (ft) ownship to the destination.
        self.distance_ownship_destination = r_do
        # Angle ownship heading relative to destination.
        self.theta_destintation_ownship = theta_do
        # ownship velocity.
        self.ownship_vel = v_do
        # intruder velocity.
        self.intruder_vel = v_i
        # Distance between intruder and ownship.
        self.distance_int_own = r_io
        # Angle intruder heading relative to ownship heading.
        self.theta_int_own_track = theta_io
        self.angle_rel_vel_neg_rel_pos = psi_io_nr_io

    # Return a string representation of a LocalState object.
    def __str__(self):
        return f"""
            distance ownship destination (r_do) = {self.distance_ownship_destination},
            angle destintation ownship (theta_do) = {self.theta_destintation_ownship},
            ownship speed (v_do) = ({self.ownship_vel}),
            distance intruder ownship (r_io) = ({self.distance_int_own}),
            angle intruder ownship track (theta_io) = {self.theta_int_own_track},
            angle of relative velocity w.r.t -(relative position) (psi_io_nr_io) = {self.angle_rel_vel_neg_rel_pos},
            intruder speed (v_i) = {self.intruder_vel}
        """

**Convert an Absolute State to a Local State**

In [18]:
def convertAbsToLocal(absolute_encounter):
    """
    Given an absolute state S, convert S to a local state L.
    """
    # Ownship position [x, y]
    ownship_pos = np.array(absolute_encounter.ownship_pos)
    # Intruder position [x, y]
    intruder_pos = np.array(absolute_encounter.intruder_pos)

    # Ownship velocity [v_x, v_y]
    ownship_vel = np.array(absolute_encounter.ownship_vel)
    # Intruder velocity [v_x, v_y]
    intruder_vel = np.array(absolute_encounter.intruder_vel)
    
    # Distance to the destination (ownship).
    destination = np.array(DESTINATION_STATE)
    dest_ownship_vector = destination - ownship_pos                 # [0,0] - [ownship_x, ownship_y].
    distance_ownship_destination = LA.norm(dest_ownship_vector)     # distance to the destination at [0,0].

    # atan2(y,x).
    theta_destintation_ownship = math.degrees(math.atan2(dest_ownship_vector[1], dest_ownship_vector[0]))
    
    psi_o = math.degrees(math.atan2(ownship_vel[0], ownship_vel[1]))    # ownship vel w.r.t y axis.
     
    speed_destination_ownship = LA.norm(destination - ownship_vel)   # speed of ownship.
    
    intruder_pos_relative_ownship = intruder_pos - ownship_pos
    distance_intruder_ownship = LA.norm(intruder_pos_relative_ownship)  # distance intruder and ownship.
    
    # intruder angle w.r.t y axis.
    theta_int_own_orig = math.degrees(math.atan2(intruder_pos_relative_ownship[0], intruder_pos_relative_ownship[1]))
    theta_intruder_own_track = theta_int_own_orig - psi_o # angle of the intruder pos w.r.t ownship's ground track.
    
    # tan^-1 (-180,180]
    if theta_intruder_own_track < -180:
        theta_intruder_own_track += 360
    
    elif theta_intruder_own_track > 180:
        theta_intruder_own_track -= 360
    
    # speed of the intruder.
    speed_intruder = LA.norm(intruder_vel)
    
    # Compute the angle between -intruder_pos_relative_ownship and intruder_vel_relative_ownship. 
    # This angle is 0 for a straight-to-collision geometry and increases in a clockwise fashion.
    intruder_vel_relative_ownship = intruder_vel - ownship_vel
    # w.r.t. the y-axis (-180, 180]
    psi_io_rel_vel = math.degrees(math.atan2(intruder_vel_relative_ownship[0], intruder_vel_relative_ownship[1]))
    
    # angle of the psi_io_rel_vel vector w.r.t. the -intruder_pos_relative_ownship vector. 
    # This angle is 0 for a straight-to-collision geometry.
    angle_rel_vel_neg_rel_pos = psi_io_rel_vel - (theta_int_own_orig - 180)
    
    if angle_rel_vel_neg_rel_pos <= -180:
        angle_rel_vel_neg_rel_pos += 360
    elif angle_rel_vel_neg_rel_pos > 180:
        angle_rel_vel_neg_rel_pos -= 360
        
    # Create local state object.
    local_state = LocalState(distance_ownship_destination,
                             theta_destintation_ownship,
                             speed_destination_ownship,
                             distance_intruder_ownship,
                             theta_intruder_own_track,
                             angle_rel_vel_neg_rel_pos,
                             speed_intruder)
    return local_state

## Is a local state in continuous space terminal?

In [19]:
def isTerminalState(state: State):
    """
    Is a Local State terminal?
    Returns a non-zero reward for a final state:

        DESTINATION_STATE_REWARD = 1.
        ABANDON_STATE_REWARD = -0.5.
        LODWC_REWARD = -0.3.

    Otherwise return 0 for a non-final states.
    """
    local_state = convertAbsToLocal(state)
    
    if local_state.distance_ownship_destination <= DESTINATION_DIST_ERROR:
        return DESTINATION_STATE_REWARD     # Close enough to the destination, reward it.
    if local_state.distance_ownship_destination > ABANDON_STATE_ERROR:
        return ABANDON_STATE_REWARD     # Too far from destination, penalty.
    if local_state.distance_int_own < DWC_DIST:
        return LODWC_REWARD     # Lost of well clear.
    
    return 0

# Monte Carlo Search Tree
## Select, Expand, Simulate, and Backpropagate
Read more about how the Monte Carlo Tree Search works: [Monte Carlo Tree Search]('https://medium.com/@quasimik/monte-carlo-tree-search-applied-to-letterpress-34f41c86e238'). This will help to understand the implementation below.

![MCTS](Images/MCTS.jpg)

In [23]:
class MCST_State:
    """
        A MCTS State represents a node on the Monte Carlo Tree (MCT).
        MCTS contains:
            1. state: A state in continuous 2D.
            2. Q: The average expected discounted sum of rewards from this node down the tree.
            3. N: The number of times this node has been selected.
            4. dirty_bit: Whether or not this node was updated on a given iteration of MCTS.
    """
    def __init__(self, state: State):
        # State properties
        self.state = state
        self.Q = 0
        self.N = 0

        # Dirty == 1 if this state was updated during simulations.
        self.dirty_bit = 0

        # Pointers to children states based on the available actions.
        self.turn_left = None
        self.turn_right = None
        self.no_turn = None
        # How many child states of this node have been expanded.
        self.visited_child_count = 0

    def updateQN(self, New_Q):
        """
        When a node on the MCT is part of a path to a new expanded node the Q value of the new
        expanded node via simulation is back-propagated to all the nodes that lead to it on the tree. This
        propagated Q value is averaged with the current Q value of the node.
        The N value is also increased by 1.
        :param New_Q: The new Q value resulting from simulation.
        """
        # First Time Update.
        if self.N == 0:
            self.Q = New_Q
        else:  # Average with current Q value.
            current_avg = self.Q
            new_avg = current_avg + ((New_Q - current_avg) / (self.N + 1))
            self.Q = new_avg

    def __str__(self):
        """
        Return a string represenation of a MCTS object.
        """
        return f'''
            STATE: {str(self.state)}
            Q: {self.Q}
            N: {self.N}
        '''

    def clean(self):
        """
        Once we process an updated node, mark it as clean.
        """
        self.dirty_bit = 0


In [24]:
class MCST:
    """
    Implementation of the procedures needed to construct a MCTS: Selection, Expansion
    Simulation, and Back-propagation.
    """
    def __init__(self, state):
        # Set the MCST initial state.
        self.root = MCST_State(state)
        self.root.N = 1
        # Every iteration there is a sequence of selections that lead to an unknown state to be expanded. Keep track
        self.visitedStatesPath = [self.root]    # Keep track of (state, action) pairs along the path to a final state.
        # Points to the last expanded node on a given iteration.
        self.lastExpandedState = self.root      # Reference to the last expanded node where simulation starts from.
        # Sequence of  stats,action,rewards tuples to be used for the model: Refer to README for more details.
        self.state_action_reward = []           # List of 3 elements tuples (state,action,reward).

    def clearStatesPath(self):
        """
        After processing the nodes, empty it for the next iteration of MCTS.
        """
        self.visitedStatesPath = [self.root]

    def getBestAction(self):
        """
        The best action to take from this node is the one with the most simulations based on UCB1.
        """
        simulations_count = [self.root.turn_left.N, self.root.no_turn.N, self.root.turn_right.N]
        action_type = ['LEFT', 'NO_TURN', 'RIGHT']
        action = action_type[simulations_count.index(max(simulations_count))]
        
        return action 

    def selection(self):
        """
        Selection step on the MCTS iteration.
        Starting  from the root, select the node with the highest UCB1 value.
        """
        mcst_node = self.root

        # We only run selection on nodes that have the 3 children expanded.
        # While a given state node has been expanded, select a child using UCB1.
        while mcst_node.visited_child_count == 3:   # LEFT, NO_TURN and RIGHT child states have been expanded.

            # Exploration term:
            c = UCB1_C

            # Explore or exploit? UCB1 formula.
            UCB1_left = mcst_node.turn_left.Q + c * math.sqrt((math.log(mcst_node.N) / mcst_node.turn_left.N))

            UCB1_right = mcst_node.turn_right.Q + c * math.sqrt((math.log(mcst_node.N) / mcst_node.turn_right.N))

            UCB1_no_turn = mcst_node.no_turn.Q + c * math.sqrt((math.log(mcst_node.N) / mcst_node.no_turn.N))

            values = [UCB1_no_turn, UCB1_left, UCB1_right]
            
            nextChildIndex = values.index(max(UCB1_no_turn, UCB1_left, UCB1_right))

            if nextChildIndex is 0:
                # Select no_turn child.
                mcst_node = mcst_node.no_turn
            elif nextChildIndex is 1:
                # Select left child.
                mcst_node = mcst_node.turn_left
            else:
                # Select right child.
                mcst_node = mcst_node.turn_right

            # Add selected node to the Visited States Path.
            self.visitedStatesPath.append(mcst_node)

        # Return a selected node that does not have all 3 children node expanded: Used for expansion().
        return mcst_node

## State Discretizers: Bin continuous data into intervals.
We cannot aim to model continuous space because it is computationally infeasible. Before we store a local state in our model, we discretize each of the features of the state to create a Discrete Local State.
We use the KBinsDiscretizer provided by sklearn: [Check it out](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html)

In [25]:
def setUpdiscretizers():
    """
    Generate and return a set of discretizers for every feature type.
    Discretizer for Distance features.
    Discretizer for Angle features.
    Discretizer for Speed features.
    :return: A set of discretizer objects.
    """

    # Generate the range of integer values for features: Defines all values to consider in discretization.
    # Depends on the MAX and MIN values set in global_constants.py
    range_distance = (np.array([[x for x in range(MIN_DISTANCE, MAX_DISTANCE + 1)]])).T
    range_angle = (np.array([[x for x in range(MIN_ANGLE, MAX_ANGLE + 1)]])).T
    range_speed = (np.array([[x for x in range(MIN_SPEED, MAX_SPEED + 1)]])).T

    # Set the number of bins to use for every feature type.
    """
        The number of bins used for every feature type directly influences the performance of the algorithm
        both in training time (larger state space)  and quality of maneuvers.
    """
    distance_bins = 121
    angle_bins = 72
    speed_bins = 57

    # Generate the discretizer objects using KBinsDiscretizer module.
    # Refer to sklearn KBinsDiscretizers documentation for discretizer types and options.
    distance_discretizer = KBinsDiscretizer(n_bins=distance_bins, encode='ordinal', strategy='uniform')
    angle_discretizer = KBinsDiscretizer(n_bins=angle_bins, encode='ordinal', strategy='uniform')
    speed_discretizer = KBinsDiscretizer(n_bins=speed_bins, encode='ordinal', strategy='uniform')

    # Fit the values for each range into bins using the discretization objects.
    distance_discretizer.fit(range_distance)
    angle_discretizer.fit(range_angle)
    speed_discretizer.fit(range_speed)

    # Compute the discrete state space size. Total of 7 features: 2 distance features, 3 angles features, 2 speed.
    space_size = (distance_bins**2) * (angle_bins**3) * (speed_bins**2)

    # Return the discretizers set to use them during training and testing.
    return [distance_discretizer, angle_discretizer, speed_discretizer, space_size]

### Discretize a local state 
[Local State Definition](#local_state) <br>
**A local state object is a 7 dimensional vector in continious space with the following features:**

- distance_ownship_destination
- theta_destintation_ownship
- ownship_vel
- intruder_vel
- distance_int_own
- theta_int_own_track
- angle_rel_vel_neg_rel_pos

**We discretize every feature to generate a 7 dimensinal discrete state object:**
<a id='discrete_local_state'></a>

In [26]:
class DiscreteLocalState:
    """
        Represents a local state after discretization.
        Every feature is turned into a corresponding bin using the discretizers.
    """
    def __init__(self, d_o_bin, d_i_o_bin, t_d_o_bin, t_i_o_bin, a_r_v_p_bin, o_v_bin, i_v_bin):
        """
            Initialize the discrete features of this discrete state using the resulting
            'bins' for every feature discretized in the continuous state space.
        """
        # Distance ownship to destination bin
        self.dis_ownship_destBIN = d_o_bin
        # Distance angle to destination bin
        self.theta_destintation_ownshipBIN = t_d_o_bin
        # Distance ownship speed bin
        self.ownship_velBIN = o_v_bin
        # Distance intruder speed bin
        self.intruder_velBIN = i_v_bin
        # Distance intruder to ownship bin
        self.dis_int_ownBIN = d_i_o_bin
        # Angle intruder heading relative to ownship heading bin.
        self.theta_int_own_trackBIN = t_i_o_bin
        # TODO: Add desc.
        self.angle_rel_vel_neg_rel_posBIN = a_r_v_p_bin
        
    def __str__(self):
        """
            A string representation of this discrete state.
        """
        return f'''
            d_o_bin = {self.dis_ownship_destBIN},
            t_d_o_bin = {self.theta_destintation_ownshipBIN},
            o_v_bin = {self.ownship_velBIN},
            i_v_bin = {self.intruder_velBIN},
            d_i_o_bin = {self.dis_int_ownBIN},
            t_d_o_bin = {self.theta_int_own_trackBIN},
            a_r_v_p_bin = {self.angle_rel_vel_neg_rel_posBIN}
        '''

    def __eq__(self, obj):
        """
            Two discrete states are the same if they share all bins for discrete
            features.
        """
        if not isinstance(obj, DiscreteLocalState):
            return False
        if self.dis_ownship_destBIN != obj.dis_ownship_destBIN:
            return False
        if self.theta_destintation_ownshipBIN != obj.theta_destintation_ownshipBIN:
            return False
        if self.ownship_velBIN != obj.ownship_velBIN:
            return False
        if self.intruder_velBIN != obj.intruder_velBIN:
            return False
        if self.dis_int_ownBIN != obj.dis_int_ownBIN:
            return False
        if self.theta_int_own_trackBIN != obj.theta_int_own_trackBIN:
            return False
        if self.angle_rel_vel_neg_rel_posBIN != obj.angle_rel_vel_neg_rel_posBIN:
            return False

        # Discrete states are the same.
        return True

## Discretize a state:
Given a [Local State](#local_state) and the discretizers for every feature type (distance, angle, speed) generate a [Discrete Local State](#discrete_local_state).

In [27]:
def discretizeLocalState(local_state, distance_discretizer, angle_discretizer, speed_discretizer):
    """
     Given a local state find the discretized versiob: Place every continuous feature into bins.
    :param local_state: A local state to discretize.
    :param distance_discretizer: The discretizer to use for distance features.
    :param angle_discretizer: The discretizer to use for angle features.
    :param speed_discretizer: The discretizer to use for speed features.
    :return: A discrete state object.
    """

    # Vector set of distance features
    LocalStateVectorDistances = [
        local_state.distance_ownship_destination,
        local_state.distance_int_own
    ]
    # Vector set of Angle features
    LocalStateVectorAngles = [
        local_state.theta_destintation_ownship,
        local_state.theta_int_own_track,
        local_state.angle_rel_vel_neg_rel_pos
    ]
    # Vector set of Speed features
    LocalStateVectorSpeeds = [
        local_state.ownship_vel,
        local_state.intruder_vel
    ]
   
    # Discretize features of the local state using the specified discretizers generated in setUpdiscretizers().
    # Returns np.arrays with the bins.
    distance_bins = distance_discretizer.transform((np.array([LocalStateVectorDistances])).T)
    angle_bins = angle_discretizer.transform((np.array([LocalStateVectorAngles])).T)
    speed_bins = speed_discretizer.transform((np.array([LocalStateVectorSpeeds])).T)

    # distance ownship to destination bin
    d_o_bin = distance_bins.T[0][0]
    # Distance intruder to ownship bin
    d_i_o_bin = distance_bins.T[0][1]

    # Angle ownship heading to destination bin.
    t_d_o_bin = angle_bins.T[0][0]
    # Angle Intruder heading relative to ownship heading bin.
    t_i_o_bin = angle_bins.T[0][1]
    # TODO: Comment.
    a_r_v_p_bin = angle_bins.T[0][2]

    # Speed bins for ownship and intruder.
    o_v_bin = speed_bins.T[0][0]
    i_v_bin = speed_bins.T[0][1]

    # Generate the discrete local state object.
    discreteLocalState = DiscreteLocalState(d_o_bin, d_i_o_bin, t_d_o_bin, t_i_o_bin, a_r_v_p_bin, o_v_bin, i_v_bin)
    
    return discreteLocalState

## Now that we  have  Discrete Local States, our model will store the following:
- A [Discrete Local State](#discrete_local_state)
- The Q value for each action from this discrete local state.
- The number of times N we have visited and updated this each action's Q value for this discrete local state in the model.



In [28]:
class StateActionQN:

    def __init__(self, d_state: DiscreteLocalState, action, Q):
        # Discrete State to be modeled by this object.
        self.discrete_state = d_state
        # The expected reward for taking a left turn.
        self.LEFT_Q = 0
        # The expected reward for going straight turn.
        self.NO_TURN_Q = 0
        # The expected reward for taking a right turn.
        self.RIGHT_Q = 0

        # Number of times I have visited this
        self.NO_TURN_N = 0
        self.LEFT_N = 0
        self.RIGHT_N = 0

        # Initial call to update the Q value of an action.
        self.update(action, Q)
    
    def getBestAction(self):
        """
        Return the action with the highest expected reward.
        :return:
        """
        actions = ['LEFT','NO_TURN','RIGHT']
        # The set of Q values for the actions.
        actionsQ = [self.LEFT_Q, self.NO_TURN_Q, self.RIGHT_Q]
        
        action = actions[actionsQ.index(max(actionsQ))]
        return action
    
    def update(self, action: str, New_Q):
        """
            Update our knowledge of this action
            from this discrete state by averaging with its previous
            Q value.
        """
        if action is '':
            return  
        
        if action is "LEFT":
            
            if self.LEFT_N == 0:
                # First Q value for this action.
                self.LEFT_Q = New_Q
        
            else:   # Average.
                current_avg = self.LEFT_Q
                new_avg = current_avg + ((New_Q - current_avg)/(self.LEFT_N+1))
                self.LEFT_Q = new_avg

            # Update number of visits to this action.
            self.LEFT_N += 1
        elif action is "RIGHT":

            if self.RIGHT_N == 0:
                # First Q value for this action.
                self.RIGHT_Q = New_Q
                
            else:   # Average.
                current_avg = self.RIGHT_Q
                new_avg = current_avg + ((New_Q - current_avg)/(self.RIGHT_N+1))
                self.RIGHT_Q = new_avg

            # Update number of visits to this action.
            self.RIGHT_N += 1

        elif action is "NO_TURN":

            if self.N == 0:
                # First Q value for this action.
                self.NO_TURN_Q = New_Q
                
            else:   # Average.
                current_avg = self.NO_TURN_Q
                new_avg = current_avg + ((New_Q - current_avg)/(self.NO_TURN_N+1))
                self.NO_TURN_Q = new_avg

            # Update number of visits to this action.
            self.NO_TURN_N += 1

    def __str__(self):
        return f'''
            [discrete_state = {self.discrete_state}]
            N(Updates) = {self.N}
            LEFT_Q = {"{:e}".format(self.LEFT_Q)},
            RIGHT_Q = {"{:e}".format(self.RIGHT_Q)},
            NO_TURN_Q = {"{:e}".format(self.NO_TURN_Q)}
        '''

    def __eq__(self, obj):
        return isinstance(obj, StateActionQN) and obj.discrete_state == self.discrete_state


**All the code described above represents the heart of the PPA algorithm. The mechanics of training and testing are implemented in the source code PPA_Learn and PPA_Test, refer to README for instructions** 